Use alpha invariance trick. This would let us do light lambdas?

push pop
copy
model vectors for pruning. Need dummy values for each decl so at least model know what to define?



In [ ]:
s = smt.Solver()
s.check()
s.model().eval(x) # I don't hate that

x

In [ ]:
from kdrag.all import *
from dataclasses import dataclass
from collections import defaultdict

@dataclass
class EGraph():
    roots : defaultdict[smt.SortRef, set[int]]
    terms: defaultdict[smt.SortRef, dict[int, smt.ExprRef]] # tuple[int, smt.SortRef]?
    uf: defaultdict[smt.SortRef, dict[int, int]]
    solver : smt.Solver
    model : smt.ModelRef
    modelvals : defaultdict[smt.SortRef, dict[int, list[int]]] # map from sort to id of value in model to list of roots with that value in model.
    def __init__(self):
        self.roots = defaultdict(set)
        self.terms = defaultdict(dict)
        self.uf = defaultdict(dict)
        self.solver = smt.Solver()
        self.solver.check()
        self.model = self.solver.model()
        self.modelvals = defaultdict(dict)
    def _find_id(self, s : smt.SortRef, eid : int) -> int:
        uf = self.uf[s]
        while eid in uf:
            eid = uf[eid]
        return eid
    def find(self, t : smt.ExprRef) -> smt.ExprRef:
        s = t.sort()
        return self.terms[s].get(self._find_id(s, t.get_id()), t)
    def _raw_union(self, s : smt.SortRef, eid1 : smt.ExprRef, eid2 : smt.ExprRef):
        id1, id2 = self._find_id(s, eid1), self._find_id(s, eid2)
        if id1 == id2:
            return id2
        else:
            self.uf[s][id1] = id2
            self.roots[s].discard(id1)
            return id2
    def union(self, t1 : smt.ExprRef, t2 : smt.ExprRef):
        id1, id2 = self.add_term(t1), self.add_term(t2) # should we add terms?
        if id1 == id2:
            return id2
        else:
            s = t1.sort()
            # Could check if constructor or other value and let win.
            # tie break towards things in terms. Then we could allow things not in terms to be in uf.
            self.uf[s][id1] = id2
            self.solver.add(t1 == t2)
            self.roots[s].discard(id1)
            return id2  
    def add_term(self, term : smt.ExprRef):
        eid = term.get_id()
        s = term.sort()
        if eid not in self.terms[s]:
            self.roots[s].add(eid)
            self.terms[s][eid] = term
            for arg in term.children():
                self.add_term(arg)
            return eid
        else:
            return self._find_id(s, eid)
    def in_terms(self, term : smt.ExprRef, strong=True) -> bool:
        s = term.sort()
        #self.solver.add(smt.Not(smt.Distinct(term, self.terms[s][term.get_id()])))
        roots = self.roots[s]
        if self._find_id(s, term.get_id()) in roots:
            return True 
        elif strong:
            # should prune based on model.
            self.solver.push()
            for eid in self.roots[s]:
                term2 = self.terms[s][eid]
                self.solver.add(term2 != term) # could do as single 
                res = self.solver.check()
                if res == smt.unsat:
                    self.solver.pop()
                    self._raw_union(s, term.get_id(), eid) # interesting. Don't put into term bank, but do cache union for later.
                    return True
            self.solver.pop()
            return False
        else:
            return False
    def rebuild(self):
        for s, roots in self.roots.items(): 
            # faster single distinct check?
            oldroots = list(roots)
            for n, eid1 in enumerate(oldroots):
                for eid2 in oldroots[:n]:
                    # Could do this better. The needed loop shrinks as we go along.
                    if self._find_id(s, eid1) != self._find_id(s, eid2):
                        self.solver.push()
                        terms = self.terms[s]
                        t1,t2 = terms[eid1], terms[eid2]
                        self.solver.add(t1 != t2)
                        res = self.solver.check()
                        if res == smt.unsat:
                            self._raw_union(s, eid1, eid2) # don't need to solver.add
                        self.solver.pop()
        assert self.solver.check() == smt.sat
        for s, roots in self.roots.items():
            for eid in roots:
                if eid not in self.terms[s]:
                    self.roots[s].discard(eid)
        self.model = self.solver.model()
    def extract(self, term : smt.ExprRef) -> smt.ExprRef: ...

x,y,z = smt.Ints("x y z")
E = EGraph()
E.union(x,y)
assert E.find(x).eq(y)

E = EGraph()
E.add_term(x)
E.add_term(y)
E.solver.add(x == y)
E.rebuild()
assert E.find(x).eq(y)

E = EGraph()
E.add_term(x)
E.solver.add(x == y)
print(E.find(y))
E.in_terms(y)
E.find(y)
# strong_find = rebuild + find

# in_terms does a mini rebuild kind of.
# May want to cache what _isn't_ in terms also... Hmm.

y


x

In [14]:
from kdrag.all import *
from dataclasses import dataclass

@dataclass
class EGraph():
    roots : set[int]
    terms: dict[int, smt.ExprRef]
    uf: dict[int, int]
    solver : smt.Solver
    def __init__(self):
        self.roots = set()
        self.terms = {}
        self.uf = {}
        self.solver = smt.Solver()
    def find_id(self, eid : int) -> int:
        while eid in self.uf:
            eid = self.uf[eid]
        return eid
    def find(self, t : smt.ExprRef) -> smt.ExprRef:
        return self.terms[self.find_id(t.get_id())]
    def raw_union(self, eid1 : smt.ExprRef, eid2 : smt.ExprRef):
        id1, id2 = self.find_id(eid1), self.find_id(eid2)
        if id1 == id2:
            return id2
        else:
            self.uf[id1] = id2
            self.roots.remove(id1)
            return id2
    def union(self, t1 : smt.ExprRef, t2 : smt.ExprRef):
        id1, id2 = self.add_term(t1), self.add_term(t2)
        if id1 == id2:
            return id2
        else:
            self.uf[id1] = id2
            self.solver.add(t1 == t2)
            self.roots.remove(id1)
            return id2  
    def add_term(self, term : smt.ExprRef):
        eid = term.get_id()
        if eid not in self.terms:
            self.roots.add(eid)
            self.terms[eid] = term
            for arg in term.children():
                self.add_term(arg)
            return eid
        else:
            return self.find_id(eid)
    def rebuild(self):
        oldroots = list(self.roots)
        for n, eid1 in enumerate(oldroots):
            for eid2 in oldroots[:n]:
                # Could do this better. The needed loop shrinks as we go along.
                if self.find_id(eid1) != self.find_id(eid2):
                    self.solver.push()
                    t1,t2 = self.terms[eid1], self.terms[eid2]
                    self.solver.add(t1 != t2)
                    res = self.solver.check()
                    if res == smt.unsat:
                        self.raw_union(eid1, eid2) # don't need to solver.add
                    self.solver.pop()




    
x,y,z = smt.Ints("x y z")
E = EGraph()
E.union(x,y)
assert E.find(x).eq(y)

E = EGraph()
E.add_term(x)
E.add_term(y)
E.solver.add(x == y)
E.rebuild()
assert E.find(x).eq(y)


In [ ]:
from kdrag.all import *
from dataclasses import dataclass

@dataclass
class EGraph():
    roots : set[int]
    terms: dict[int, smt.ExprRef]
    explain : dict[int, object]
    uf: dict[int, int]
    solver : smt.Solver
    def __init__(self):
        self.roots = set()
        self.terms = {}
        self.uf = {}
        self.solver = smt.Solver()
    def find_id(self, eid : int) -> int:
        while eid in self.uf:
            eid = self.uf[eid]
        return eid
    def find(self, t : smt.ExprRef) -> smt.ExprRef:
        return self.terms[self.find_id(t.get_id())]
    def raw_union(self, eid1 : smt.ExprRef, eid2 : smt.ExprRef):
        id1, id2 = self.find_id(eid1), self.find_id(eid2)
        if id1 == id2:
            return id2
        else:
            self.uf[id1] = id2
            self.roots.remove(id1)
            return id2
    def union(self, t1 : smt.ExprRef, t2 : smt.ExprRef):
        id1, id2 = self.add_term(t1), self.add_term(t2)
        if id1 == id2:
            return id2
        else:
            self.uf[id1] = id2
            self.solver.add(t1 == t2)
            self.roots.remove(id1)
            return id2  
    def add_term(self, term : smt.ExprRef):
        eid = term.get_id()
        if eid not in self.terms:
            self.roots.add(eid)
            self.terms[eid] = term
            for arg in term.children():
                self.add_term(arg)
            return eid
        else:
            return self.find_id(eid)
    def rebuild(self):
        oldroots = list(self.roots)
        for n, eid1 in enumerate(oldroots):
            for eid2 in oldroots[:n]:
                # Could do this better. The needed loop shrinks as we go along.
                if self.find_id(eid1) != self.find_id(eid2):
                    self.solver.push()
                    t1,t2 = self.terms[eid1], self.terms[eid2]
                    self.solver.add(t1 != t2)
                    res = self.solver.check()
                    if res == smt.unsat:
                        self.raw_union(eid1, eid2) # don't need to solver.add
                    self.solver.pop()

keying on sort makes it more complex a little, but also prunes some possible comparisons (which are nonsenical anyway)

In [ ]:
@dataclass
class EGraph():
    canon_terms : dict[int, smt.ExprRef]
    terms : dict[int, smt.ExprRef]
    uf : dict[int, int]
    solver : smt.Solver
    #models : list[m]
    #sigs : dict[tuple[Value,...], list[TermRef]]
    #explain :   #  dict[(int,int), unsat cores]

    def add_term(self, t):
        if t.get_id() not in self.terms:
            self.terms[t.get_id()] = t
            for c in t.children():
                self.add_term(c)
    def term_matches(self, t): # guard=None
        #self.solver.push()
        #self.solver.add(cond)
        # prune by models?
        for s in self.canon_terms.values():
            self.solver.push()
            self.solver.add(t != s)
            res = self.solver.check()
            self.solver.pop()
            if res == smt.unsat:
                yield s

    def union(self, e1, e2):
        self.solver.add(e1 == e2)
        # orient by smaller depth or sexpr size. Kind of could get some sharing action
        # maybe immediately remove e1 e2
        """
        e1 = self.find(e1)
        e2 = self.find(e2)
        if e1.eq(e2):
            return e1
        else:
            del self.canon_terms[e1.get_id()]
            self.terms[e1.get_id()] = e1
        """

    def rebuild(self):
        for t in self.canon_terms.values():
            for s in self.matches(t):
                self.union(t, s)





SMT as a theory. SMT solvers themselves present the interface of a theory.

Cody says _is_ interpolation.
If whole theory is unsat  A /\ B, there exists C that only has shared symbols such that A /\ C and B /\ C are unsat.



Nelson Oppen style combination of theories vs Shostak.
Nelson oppen theories

https://web.stanford.edu/class/cs357/lecture11.pdf


In [ ]:
class Theory():
    def register_term(self, term) -> V:
    def assert_eq(self, a, b) -> list[tuple[V,V]]: # propagates
    def is_eq(self, a, b) -> Option[bool]:  # True if justified true, false if can't be true, None if both possible.

    